<a href="https://colab.research.google.com/github/sarahlintang/Simple-Deep-Learning-Indonesia-Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fawza\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fawza\AppData\Roaming\nltk_data...


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json', 'r', encoding='utf-8').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('omw-1.4')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fawza\AppData\Roaming\nltk_data...


1698 documents
241 classes ['administrasi_sidang_biaya_perkara', 'administrasi_sidang_hak_persidangan', 'administrasi_sidang_hitung_perkara', 'administrasi_sidang_panjar_habis', 'administrasi_sidang_persyaratan', 'administrasi_sidang_pertimbangan_panjar', 'aduan_layanan_prosedur', 'ecourt_batas_waktu_bayar', 'ecourt_bayar', 'ecourt_bayar_titip', 'ecourt_call_center', 'ecourt_daftar_tanpa_email', 'ecourt_data', 'ecourt_definisi', 'ecourt_dokumen_daftar_perkara', 'ecourt_e-filling', 'ecourt_e-litigation', 'ecourt_e-payment', 'ecourt_e-summons', 'ecourt_edit_dokumen', 'ecourt_email_aktivasi', 'ecourt_info_sisa_panjar', 'ecourt_info_tambahan_panjar', 'ecourt_konfirmasi_salah_bayar', 'ecourt_kredit', 'ecourt_kuasa_lebih_dari_satu', 'ecourt_layanan', 'ecourt_legal_daftar', 'ecourt_non_advokat', 'ecourt_panggilan_elektronik_biaya', 'ecourt_panggilan_elektronik_email', 'ecourt_panggilan_luar_negeri', 'ecourt_panggilan_pertama_tergugat', 'ecourt_pembayaran_bank', 'ecourt_pemula', 'ecourt_pengaj

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\fawza\AppData\Local\Temp\ipykernel_16768\74795701.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_keras_dnn.h5', hist)

print("model created")

Epoch 1/200
340/340 [==============================] - 2s 3ms/step - loss: 5.2897 - accuracy: 0.0925
Epoch 2/200
340/340 [==============================] - 1s 3ms/step - loss: 4.9187 - accuracy: 0.0966
Epoch 3/200
340/340 [==============================] - 1s 4ms/step - loss: 4.6334 - accuracy: 0.1084
Epoch 4/200
340/340 [==============================] - 1s 3ms/step - loss: 4.3673 - accuracy: 0.1266
Epoch 5/200
340/340 [==============================] - 1s 3ms/step - loss: 4.0722 - accuracy: 0.1614
Epoch 6/200
340/340 [==============================] - 1s 3ms/step - loss: 3.7419 - accuracy: 0.2044
Epoch 7/200
340/340 [==============================] - 1s 3ms/step - loss: 3.4323 - accuracy: 0.2132
Epoch 8/200
340/340 [==============================] - 1s 3ms/step - loss: 3.1251 - accuracy: 0.2580
Epoch 9/200
340/340 [==============================] - 1s 3ms/step - loss: 2.9295 - accuracy: 0.2939
Epoch 10/200
340/340 [==============================] - 1s 3ms/step - loss: 2.7273 - accura